In [1]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import renom as rm
from renom.cuda.cuda import set_cuda_active
set_cuda_active(False)

In [2]:
filename = "./isolet1+2+3+4.data"
labels = []
X = []
y = []

def make_label_idx(filename):
    labels = []
    for line in open(filename, "r"):
        line = line.rstrip()
        label = line.split(",")[-1]
        labels.append(label)
    labels = list(set(labels))
    return list(set(labels))

labels = make_label_idx(filename)
labels = sorted(labels, key=lambda d:int(d.replace(".","").replace(" ","")))

In [3]:
for line in open(filename,"r"):
    line = line.rstrip()
    label = labels.index(line.split(",")[-1])
    features = list(map(float,line.split(",")[:-1]))
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("X_train:{}, y_train:{}, X_test:{}, y_test:{}".format(X_train.shape, y_train.shape,
                                                            X_test.shape, y_test.shape))
lb = LabelBinarizer().fit(y)
labels_train = lb.transform(y_train)
labels_test = lb.transform(y_test)
print("labels_train:{}, labels_test:{}".format(labels_train.shape, labels_test.shape))

X_train:(4990, 617), y_train:(4990,), X_test:(1248, 617), y_test:(1248,)
labels_train:(4990, 26), labels_test:(1248, 26)


In [4]:
output_size = len(labels)
sequential = rm.Sequential([
    rm.Dense(100),
    rm.Relu(),
    rm.Dense(50),
    rm.Relu(),
    rm.Dense(output_size)
])

In [5]:
epoch = 20
batch_size = 128
N = len(X_train)
optimizer = rm.Adam(lr=0.01)
for i in range(epoch):
    perm = np.random.permutation(N)
    loss = 0
    for j in range(0, N//batch_size):
        train_batch = X_train[perm[j*batch_size : (j+1)*batch_size]]
        response_batch = labels_train[perm[j*batch_size : (j+1)*batch_size]]
        with sequential.train():
            l = rm.softmax_cross_entropy(sequential(train_batch), response_batch)
        grad = l.grad()
        grad.update(optimizer)
        loss += l.as_ndarray()
    train_loss = loss / (N//batch_size)
    test_loss = rm.softmax_cross_entropy(sequential(X_test), labels_test).as_ndarray()
    print("epoch:{:03d}, train_loss:{:.4f}, test_loss:{:.4f}".format(i, float(train_loss), float(test_loss)))

epoch:000, train_loss:1.3317, test_loss:0.4834
epoch:001, train_loss:0.3664, test_loss:0.3139
epoch:002, train_loss:0.2488, test_loss:0.2133
epoch:003, train_loss:0.1843, test_loss:0.3111
epoch:004, train_loss:0.1542, test_loss:0.2365
epoch:005, train_loss:0.1022, test_loss:0.1880
epoch:006, train_loss:0.1007, test_loss:0.1825
epoch:007, train_loss:0.0902, test_loss:0.1663
epoch:008, train_loss:0.0653, test_loss:0.1451
epoch:009, train_loss:0.0524, test_loss:0.3906
epoch:010, train_loss:0.0692, test_loss:0.1800
epoch:011, train_loss:0.0621, test_loss:0.2338
epoch:012, train_loss:0.1053, test_loss:0.2653
epoch:013, train_loss:0.0978, test_loss:0.2123
epoch:014, train_loss:0.0598, test_loss:0.2020
epoch:015, train_loss:0.0470, test_loss:0.2419
epoch:016, train_loss:0.0420, test_loss:0.2790
epoch:017, train_loss:0.0339, test_loss:0.2289
epoch:018, train_loss:0.0206, test_loss:0.2019
epoch:019, train_loss:0.0132, test_loss:0.2602
